# Import modules

In [3]:
import numpy as np
import json

from keras.models import Sequential
from keras.layers import LSTM, Dense, Embedding, GRU, Input, TimeDistributed, Activation
from keras.optimizers import RMSprop
from sklearn.preprocessing import OneHotEncoder
from keras.callbacks import LambdaCallback


# Get data

In [4]:
with open("../dictionaries/first-names.json") as f:
    rawData = (json.load(f))
    
# Add escape character '.'
for i in range(len(rawData)):
    rawData[i] += "."
                           
# rawData = np.array(rawData)
# print(rawData.shape)

# One hot encode data

In [5]:
import string
alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','.']
def string_vectorizer(strng, nMax):
    vector = [[0 if char != letter else 1 for char in alphabet] 
                  for letter in strng]
    # add padding
    while (len(vector) < nMax):
        vector.append([0 for char in alphabet])
    return vector

In [6]:
nMax = len(max(rawData, key=len))
char_dim = 27
onehot = []
print('max characters is', nMax)

for i in rawData:
    onehot.append(string_vectorizer(i,nMax))
dataClean = np.array(onehot)
print(dataClean.shape)

max characters is 15
(4946, 15, 27)


In [7]:
print(dataClean.shape)

(4946, 15, 27)


# Train model

In [42]:
original_dim = (dataClean.shape[1], dataClean.shape[2])
model = Sequential()
# Simple model
model.add(LSTM(100, return_sequences=True))

model.add(Dense(char_dim, activation='softmax'))

# optimizer = RMSprop(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(dataClean,dataClean, batch_size=256, epochs=3000, callbacks=[name_generator])


Epoch 1/3000
4946/4946 [==============================] - 1s 225us/step - loss: 1.5206
Names generated after epoch 0:
nn.
n.
.
nn.
n.
Epoch 2/3000
4946/4946 [==============================] - 1s 153us/step - loss: 1.3272
Epoch 3/3000
4946/4946 [==============================] - 1s 159us/step - loss: 1.2028
Epoch 4/3000
4946/4946 [==============================] - 1s 156us/step - loss: 1.1271
Epoch 5/3000
4946/4946 [==============================] - 1s 148us/step - loss: 1.0330
Epoch 6/3000
4946/4946 [==============================] - 1s 152us/step - loss: 0.9053
Names generated after epoch 5:
aiiaaaaaaaaanaa
aiaaaainaaaaana
iialanaaaaaaaaa
aiaaaaaaaaaaaaa
aaaaaaaaaaaaaaa
Epoch 7/3000
4946/4946 [==============================] - 1s 157us/step - loss: 0.7514
Epoch 8/3000
4946/4946 [==============================] - 1s 156us/step - loss: 0.5958
Epoch 9/3000
4946/4946 [==============================] - 1s 152us/step - loss: 0.4569
Epoch 10/3000
4946/4946 [==============================] - 

KeyboardInterrupt: 

# Generate words

In [14]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]
print(get_available_devices()) 


['/device:CPU:0', '/device:XLA_CPU:0', '/device:GPU:0', '/device:XLA_GPU:0']


In [41]:
def generate_name_loop(epoch, _):
    if epoch % 5 == 0:
        
        print('Names generated after epoch %d:' % epoch)

        for i in range(5):
            print(make_name(model))
        
        
      
name_generator = LambdaCallback(on_epoch_end = generate_name_loop)


def vec_to_string(vec):
    print(vec.shape)
    maxList = vec.argmax(axis=1)
    print(maxList)

    
def make_name(model):
    alphabet = ['a','b','c','d','e','f','g','h','i','j','k','l','m','n','o','p','q','r','s','t','u','v','w','x','y','z','.']

    name = []
    x = np.random.random((1, nMax, char_dim))
    end = False
    i = 0
    while end == False and i < nMax:
        
        probs = (model.predict(x)[0,i])
        argmax = (np.argmax(probs))
        name += alphabet[argmax]
        if argmax == len(alphabet)-1:
            end=True
            break
        i+=1
    return ''.join(name)

print(make_name(model))
#     print(''.join(name))
# for i in range(10):
#     myName = ''
#     while (len(myName) < 5):  
#         myName = make_name(model)
#     print(myName, len(myName))

eaae.
